In [2]:
%matplotlib inline
%config InlineBackend.figure_format = 'svg'

import scqubits as scq
from scqubits import HilbertSpace, InteractionTerm, ParameterSweep

import numpy as np

np.set_printoptions(threshold=10, edgeitems=2)

In [3]:
# Define HilbertSpace object: two transmons coupled to an oscillator


tmon1 = scq.TunableTransmon(
    EJmax=40.0,
    EC=0.2,
    d=0.1,
    flux=0.23,
    ng=0.3,
    ncut=40,
    truncated_dim=3,     # after diagonalization, we will keep 3 levels
    id_str="tmon1"       # optional, used for referencing from within
                         # ParameterSweep or HilbertSpace
)

tmon2 = scq.TunableTransmon(
    EJmax=15.0,
    EC=0.15,
    d=0.2,
    flux=0.0,
    ng=0.0,
    ncut=30,
    truncated_dim=3,
    id_str="tmon2"
)

resonator = scq.Oscillator(
    E_osc=4.5,
    truncated_dim=4    # up to 3 photons (0,1,2,3)
)

hilbertspace = scq.HilbertSpace([tmon1, tmon2, resonator])


g1 = 0.1  # coupling resonator-CPB1 (without charge matrix elements)
g2 = 0.2  # coupling resonator-CPB2 (without charge matrix elements)

hilbertspace.add_interaction(
    g_strength = g1,
    op1 = tmon1.n_operator,
    op2 = resonator.creation_operator,
    add_hc = True,
    id_str="tmon1-resonator"  # optional keyword argument
)

hilbertspace.add_interaction(
    g_strength = g2,
    op1 = tmon2.n_operator,
    op2 = resonator.creation_operator,
    add_hc = True,
    id_str="tmon2-resonator"  # optional keyword argument
)

In [4]:
# Set up parameter name and values
pname1 = 'flux'  
flux_vals = np.linspace(0.0, 2.0, 171)
pname2 = 'ng'
ng_vals = np.linspace(-0.5, 0.5, 49)

# combine into a dictionary
paramvals_by_name = {pname1: flux_vals, pname2: ng_vals}


area_ratio = 1.2

def update_hilbertspace(flux, ng):  # function that defines how Hilbert space components are updated
    tmon1.flux = flux
    tmon2.flux = area_ratio * flux
    tmon2.ng = ng

# dictionary with information on which subsystems are affected by changing parameters    
subsys_update_info = {pname1: [tmon1, tmon2],
                      pname2: [tmon2]}

    
# create the ParameterSweep    
sweep = ParameterSweep(
    hilbertspace=hilbertspace,
    paramvals_by_name=paramvals_by_name,
    update_hilbertspace=update_hilbertspace,
    evals_count=20,
    subsys_update_info=subsys_update_info,
    num_cpus=4
)

Parallel compute bare eigensys for subsystem tmon1 [num_cpus=4]

Parallel compute bare eigensys for subsystem tmon2 [num_cpus=4]

Parallel compute bare eigensys for subsystem Oscillator_1 [num_cpus=4]

Parallel compute dressed eigensys [num_cpus=4]

## Global objects vs. `deepcopy`

In the above code, `update_hilbertspace` directly manipulates the two transmon
instances via global variables. It is generally considered better programming style to avoid such use of global variables.
To facilitate this, `update_hilbertspace` may be defined with an additional first argument that takes in the `ParameterSweep` object itself. This way, all `HilbertSpace` constituents can be accessed via

```<ParameterSweep>.hilbertspace[<id_str>]```:

In [5]:
def update_hilbertspace(param_sweep, flux, ng):  # function that defines how Hilbert space components are updated
    param_sweep.hilbertspace["tmon1"].flux = flux
    param_sweep.hilbertspace["tmon2"].flux = area_ratio * flux
    param_sweep.hilbertspace["tmon2"].ng = ng

This way of updating is of particular interest when using the `deepcopy` option
discussed next.

#### State after the sweep and using `deepcopy`

By default, the `HilbertSpace` object and its constituents will be left by `ParameterSweep` in a state reached with the last evaluation involved in the sweep. (When multiprocessing, this final state may not be easy to predict.)

Alternatively, the `deepcopy` option can be used to
1. disconnect all global ``HilbertSpace` constituents from the sweep,and
2. restore the `HilbertSpace` object stored internally with the `ParameterSweep` to the parameters prior to the sweep.

Without use of `deepcopy`, there is a good chance that, for example, the offset
charge of `tmon2` is now different than prior to the sweep.
For `deepcopy=True` the global instances are disconnected from the sweep
and remain untouched. The `HilbertSpace` object interior to `sweep` is restored to its
original state, but remains a copy separate from the `HilbertSpace` object used to
initialize `ParameterSweep`.

.. note:
    When using `deepcopy=True`, the original state of `HilbertSpace` objects
    is restored. With `deepcopy` enabled,
    `update_hilbertspace` **must** be based on manipulating the objects interior to
    `ParameterSweep`. (Changing the global objects will have no effect in this case.)

